**Introduction**


**If You Like this notebook please consider to upvote it**


This Dataset contains the information about Titanic ship and included 891Rows and 12 columns.In this notebook I try to find the best ML model to predict which passengers survived the Titanic shipwreck. Variable Notes:

pclass: A proxy for socio-economic status (SES)(Ticket class) 1 = 1st, 2 = 2nd, 3 = 3rd

1st = Upper

2nd = Middle

3rd = Lower

age:Age in years

Sibsp: The dataset defines family relations in this way...(number of siblings / spouses aboard the Titanic)

Sibling = brother, sister, stepbrother, stepsister

Spouse = husband, wife (mistresses and fiancés were ignored)

Parch: The dataset defines family relations in this way...(number of parents / children aboard the Titanic)

Parent = mother, father

Child = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them.

Survival:0 = No, 1 = Yes

Sex:Sex

Ticket:Ticket number

Fare:Passenger fare

Cabin:Cabin number

Embarked:Port of Embarkation(C = Cherbourg, Q = Queenstown, S = Southampton)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
%matplotlib inline 
import seaborn as sns

**Reading Train and Test data from the datatset**

In [ ]:
train = pd.read_csv("../input/titanic-solution-for-beginners-guide/train.csv")
test = pd.read_csv("../input/titanic-solution-for-beginners-guide/test.csv")
train.head()
test.head()

**Missing values**

In [ ]:
train.isnull().sum()

**Dropping not so cordinating columns with survived column**

In [ ]:
train.drop(['PassengerId','Name','Ticket','Cabin'],axis='columns',inplace=True)
test.drop(['PassengerId','Name','Ticket','Cabin'],axis='columns')
train.head(2)

In [ ]:
train.dtypes

**we need to change Sex and Embarked categorical columns into numeric colums**

In [ ]:
train.describe()

In [ ]:
dict={}
for i in train.columns:
    dict[i]=train[i].value_counts().shape[0]
pd.DataFrame(dict,index=['unique']).transpose()

**Categorical Features: Sex, Parch, Embarked**

**Ordinal Features: Pclass**

**Continous Features: age, fare**

**Discrete Feature: SibSip**

**Target Feature: Survived**

**categorical feature= sex,parch,embarked,pclass,sibsp,survived**

**continous feature=age,fare**

In [ ]:
fig = plt.figure(figsize=(20,12))
gs = fig.add_gridspec(2,4)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])

sns.countplot('Survived', data=train, ax=ax0).set(xlabel='Survived')
sns.countplot('Parch', data=train, ax=ax1).set(xlabel='parch')
sns.countplot('Embarked', data=train, ax=ax2).set(xlabel='Embarked')
sns.countplot('SibSp', data=train, ax=ax3).set(xlabel='SibSp')
sns.countplot('Sex', data=train, ax=ax4).set(xlabel='sex')
sns.countplot('Pclass', data=train, ax=ax5).set(xlabel='Pclass')


In [ ]:
fig=plt.figure(figsize=(15,12))
gs=fig.add_gridspec(1,2)
axA=fig.add_subplot(gs[0,0])
axB=fig.add_subplot(gs[0,1])

sns.kdeplot(train['Age'],data=train,ax=axA,fill=True,color='orange').set(xlabel='Age')
sns.kdeplot(train['Fare'],data=train,ax=axB,fill=True,color='red').set(xlabel='Fare')
plt.text(180, 0.0232,'Age curve is seems likely to have guassian curve whereas Fare curve isnt a guassian curve so we need to transform',fontsize='22')

**Univariate Analysis with respect to Survived Column(Target)**

In [ ]:
# blue is died and yellow is lived
fig=plt.figure(figsize=(20,12))
gs=fig.add_gridspec(2,3)
ax0=fig.add_subplot(gs[0,0])
ax1=fig.add_subplot(gs[0,1])
ax2=fig.add_subplot(gs[0,2])
ax3=fig.add_subplot(gs[1,0])
ax4=fig.add_subplot(gs[1,1])
ax5=fig.add_subplot(gs[1,2])

sns.kdeplot(train['Age'],data=train,hue='Survived',ax=ax0,fill=True,color='red').set(xlabel='Age')
sns.countplot('Sex',data=train,hue='Survived',ax=ax1).set(xlabel='Sex')
sns.countplot('Parch',data=train,hue='Survived',ax=ax2).set(xlabel='Parch')
sns.countplot('Embarked',data=train,hue='Survived',ax=ax3).set(xlabel='embarked')
sns.countplot('Pclass',data=train,hue='Survived',ax=ax4).set(xlabel='pclass')
sns.countplot('SibSp',data=train,hue='Survived',ax=ax5).set(xlabel='sibsp')

In [ ]:
# for heatmap withour confusion matrix we use this special feature which is .corr()
matrix=np.triu(train.corr()) # for half heatmap we use np.triu and use mask inside heatmap
plt.figure(figsize=(15,9))
sns.heatmap(train.corr(),annot=True,mask=matrix)

**heatmap helps to corelate the features with each other**


**Pclass has a good negative correlation with Fare. Its kinda obvious that as Pclass increases Fare will decrease(Third Class will cost less than First class).**

**Fare also has minor positive correlation with Survived. Again people who paid more had more chances of survival.**

**SibSp has good positive correlation with Parch. We can concatenate these together into a single family column later on**

In [ ]:
fig = plt.figure(figsize=(18,10))
gs = fig.add_gridspec(2,2)
gs.update(wspace=0.5, hspace=0.25)
axC = fig.add_subplot(gs[0,0])
# axD = fig.add_subplot(gs[1,0])
axA = fig.add_subplot(gs[0,1])
axB = fig.add_subplot(gs[1,1])

sns.boxenplot(x="Pclass",y="Age",data=train, palette=['#FD151B' ,'#97D2FB' ,'#F9C846' ,'#437F97'], ax=axA)

sns.boxenplot(x="Survived",y="Age",data=train, palette=['#FD151B' ,'#97D2FB' ,'#F9C846' ,'#437F97'], ax=axB)

sns.boxenplot(x='Sex',y='Age',data=train,palette=['#FD151B' ,'#97D2FB' ,'#F9C846' ,'#437F97'],ax=axC)

**Pclass 1 had people mostly had people aged between 30 and 50 years. It might be because the more aged people might have better jobs and finances to afford first class.**

**Pclass 2 had people aged between 20-40 more**

**Pclass 3 had more people age between 20-30**

**Age column has significant outliers as observed from the plots**

**Sex columns has significant outliers as observed from the plots**

**Test data**

In [ ]:
test.describe()

In [ ]:
dict={}
for i in test.columns:
    dict[i]=test[i].value_counts().shape[0]
pd.DataFrame(dict,index=['unique']).transpose()

In [ ]:
test.isnull().sum()

In [ ]:
test.drop(['PassengerId','Ticket','Cabin','Name'],axis='columns',inplace=True)
test.head()

In [ ]:
fig = plt.figure(figsize=(20,12))
gs = fig.add_gridspec(2,4)
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])

sns.countplot('Parch', data=test, ax=ax1).set(xlabel='parch')
sns.countplot('Embarked', data=test, ax=ax2).set(xlabel='Embarked')
sns.countplot('SibSp', data=test, ax=ax3).set(xlabel='SibSp')
sns.countplot('Sex', data=test, ax=ax4).set(xlabel='sex')
sns.countplot('Pclass', data=test, ax=ax5).set(xlabel='Pclass')



In [ ]:
fig=plt.figure(figsize=(15,12))
gs=fig.add_gridspec(1,2)
axA=fig.add_subplot(gs[0,0])
axB=fig.add_subplot(gs[0,1])

sns.kdeplot(test['Age'],data=test,ax=axA,fill=True,color='orange').set(xlabel='Age')
sns.kdeplot(test['Fare'],data=test,ax=axB,fill=True,color='red').set(xlabel='Fare')

In [ ]:
print('training data')
print('median age for people who are in first class',train[train['Pclass']==1]['Age'].median())
print('median age for people who are in second class',train[train['Pclass']==2]['Age'].median())
print('median age for people who are in third class',train[train['Pclass']==3]['Age'].median())

In [ ]:
print('testing data')
print('median age for people who are in first class',test[test['Pclass']==1]['Age'].median())
print('median age for people who are in second class',test[test['Pclass']==2]['Age'].median())
print('median age for people who are in third class',test[test['Pclass']==3]['Age'].median())

In [ ]:
def input_age_train(columns):
    Age=columns[0]
    Pclass=columns[1]
    
    if pd.isnull(Age):
        
        if Pclass==1:
            return 37
        elif Pclass==2:
            return 29
        else:
            return 24
    else:
        return Age

In [ ]:
train['Age']=train[['Age','Pclass']].apply(input_age_train,axis=1)

In [ ]:
def input_age_test(columns):
    Age=columns[0]
    Pclass=columns[1]
    
    if pd.isnull(Age):
        
        if Pclass==1:
            return 42
        elif Pclass==2:
            return 26
        else:
            return 24
    else:
        return Age

In [ ]:
test['Age']=test[['Age','Pclass']].apply(input_age_test,axis=1)

In [ ]:
test.isnull().sum()

In [ ]:
test[test['Fare'].isnull()]

In [ ]:
print('median of S class is ',test[test['Pclass']==3]['Fare'].mean())

In [ ]:
test['Fare']=test['Fare'].fillna(12.46)

In [ ]:
test.isnull().sum()

In [ ]:
fig=plt.figure(figsize=(15,9))
gs=fig.add_gridspec(1,2)
ax0=fig.add_subplot(gs[0,0])
ax1=fig.add_subplot(gs[0,1])

sns.kdeplot(train['Fare'],data=train,ax=ax0,fill=True,color='red').set(xlabel='train_fare')
sns.kdeplot(test['Fare'],data=test,ax=ax1,fill=True,color='orange').set(xlabel='train_fare')

In [ ]:
train["Fare"] = train["Fare"].map(lambda i: np.log(i) if i > 0 else 0)
test["Fare"] = test["Fare"].map(lambda i: np.log(i) if i > 0 else 0)

In [ ]:
fig=plt.figure(figsize=(10,5))
gs=fig.add_gridspec(1,2)
ax0=fig.add_subplot(gs[0,0])
ax1=fig.add_subplot(gs[0,1])

sns.kdeplot(train['Fare'],data=train,ax=ax0,fill=True,color='red').set(xlabel='train_fare')
sns.kdeplot(test['Fare'],data=test,ax=ax1,fill=True,color='orange').set(xlabel='train_fare')

In [ ]:
test.head()

In [ ]:
print(test['Pclass'].unique())
print(test['SibSp'].unique())
print(test['Parch'].unique())
print(test['Embarked'].unique())

In [ ]:
train['Age']=pd.qcut(train['Age'],10,duplicates='drop')
test['Age']=pd.qcut(test['Age'],10,duplicates='drop')

In [ ]:
fig=plt.figure(figsize=(15,9))
gs=fig.add_gridspec(1,1)
ax0=fig.add_subplot(gs[0,0])

sns.countplot(train['Age'],data=train,hue='Survived',ax=ax0).set(xlabel='Age wrt survived')

**people in range 20-24 of Age survived and died the most**

**adding sibsp and parch in one column (family size)**

In [ ]:
train['familysize']=train['SibSp']+train['Parch']+1
test['familysize']=test['SibSp']+test['Parch']+1

In [ ]:
train.loc[(train['familysize']==1),'familysize']=0 #alone
train.loc[(train['familysize']>1) & (train['familysize']<=4),'familysize']=1 #samllfamily
train.loc[(train['familysize']>4) & (train['familysize']<=6),'familysize']=2 #mediumfsmily
train.loc[(train['familysize']>6),'familysize']=3 #largefsmily

In [ ]:
test.loc[(test['familysize']==1),'familysize']=0 #alone
test.loc[(test['familysize']>1) & (test['familysize']<=4),'familysize']=1 #samllfamily
test.loc[(test['familysize']>4) & (test['familysize']<=6),'familysize']=2 #mediumfsmily
test.loc[(test['familysize']>6),'familysize']=3 #largefsmily

In [ ]:
train.drop(['SibSp','Parch'],axis='columns',inplace=True)
test.drop(['SibSp','Parch'],axis='columns',inplace=True)

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
train['Age']=LabelEncoder().fit_transform(train['Age'])
test['Age']=LabelEncoder().fit_transform(test['Age'])

In [ ]:
c=train[['Sex','Embarked']]
dummies=pd.get_dummies(c)
dummies.head(10)

In [ ]:
dummies1=pd.get_dummies(train['Pclass'])
dummies1.head()

In [ ]:
dummies2=pd.concat([dummies,dummies1],axis=1)
dummies2.head()

In [ ]:
dummies2.rename(columns={1:'Pclass_1',2:'Pclass_2',3:'Pclass_3'},inplace=True)

In [ ]:
dummies2.head()

In [ ]:
train_new=pd.concat([dummies2,train],axis=1)
train_new

In [ ]:
train_new.drop(['Pclass','Sex','Embarked'],axis=1,inplace=True)
train_new.head()

**Models**

In [ ]:
from sklearn.model_selection import train_test_split
X=train_new.drop('Survived',axis=1)
y=train_new['Survived']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

****K-nearest neighbour****

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=34)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

**Random Forest Classification**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

**SVM**

In [ ]:
from sklearn.svm import SVC
model=SVC()
model.fit(X_train,y_train)
model.score(X_test,y_test)

**HyperTuning of RandomForest**

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.score(X_test,y_test)